<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Investigate_Gapminders_WDI/Investigate_R_and_D_Expenditures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Investigate Gapminder's World Development Indicators

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#questions">Questions</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#limitation">Limitation</a></li>
</ul>

# Introduction <a id='intro'></a>

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

**[Gapminder](https://www.gapminder.org/)** has collected a lot of information about how people live their lives in different countries, tracked across the years, and on a number of different indicators. The complete datasets with hundreds of indicators are available in GitHub repositories, one of which is World Development Indicators (WDI). The data is organized in loose CSV files which can be consumed by any spreadsheet software. More efficient way of consuming data is by using [DDF data model](https://open-numbers.github.io/ddf.html).

DDF is used to define datasets. A dataset is a body of coherent, related data that is composed of separate elements, but can be manipulated as one unit by a computer. Each DDF dataset must have Concepts and may have DataPoints, Entities, Metadata, or Synonyms, whereas:

- **Concepts** contain information about the variables in the data set, i.e. concept properties with simple key (concept = the column-header in a tabular format).
- **DataPoints** contain multidimensional data, i.e. indicators with composite keys (dimensions).
- **Entities** contain single-dimensional data, i.e. entity properties with simple keys (entity).

What datapoints, entities, and concepts to find in what files is defined in the schema and resources sections of a `datapackage.json` file. This is the entry point for any machine that wants to explore the data set:

In [1]:
# Import dependencies
import requests
import json
import pandas as pd

In [8]:
# Set path to data repozitory
path = 'https://raw.githubusercontent.com/open-numbers/ddf--open_numbers--world_development_indicators/master/'
# Read datapackage.json with data descriptions
datapkg = requests.get(path + 'datapackage.json').json()

# Print key information about a data set
for k,v in datapkg.items():
  if type(v) == str or len(v) < 3:
    print(k, ' :\t', v)
  else:
    print(k, ' : section contains ', len(v), 'elements.')

name  :	 ddf--gapminder--world_development_indicators
language  :	 {'name': 'English', 'id': 'en'}
title  :	 Gapminder's World Development Indicators
description  :	 Gapminder's World Development Indicators
author  :	 Gapminder
license  :	 MIT
created  :	 2021-08-22T12:15:33.269367+00:00
translations  :	 []
version  :	 0.0.1
resources  : contain(s)  2402 elements.
ddfSchema  : contain(s)  4 elements.


This data analysis investigates two dependent variables (indicators):
- Research and development expenditure (% of GDP)
- GDP growth (annual %)

Key description of Research and development expenditure (concept `gb_xpd_rsdv_gd_zs`) and GDP growth (`ny_gdp_mktp_kd_zg`) is provided by `ddf--concepts--continuous.csv` file:

In [3]:
# Load descriptions of continuous concepts
wdic_cont = pd.read_csv(path + 'ddf--concepts--continuous.csv', index_col='concept')

# Define indicators for further analysis
indicators = ['gb_xpd_rsdv_gd_zs', 'ny_gdp_mktp_kd_zg']

# Print key information about indicators
import textwrap # ensures more readable text rendering in ipynb, html, as well as pdf
for i in indicators:
  print('\nConcept:', i)
  print('==========================')
  for v in ['name', 'long_definition', 'statistical_concept_and_methodology', 'source']:
    text = v.replace('_', ' ').capitalize() + ' : ' + wdic_cont.at[i,v]
    for line in textwrap.wrap(text, width=80):
      print(line.replace('\\n', '\n'))
    print()



Concept: gb_xpd_rsdv_gd_zs
Name : Research and development expenditure (% of GDP)

Long definition : Gross domestic expenditures on research and development (R&D),
expressed as a percent of GDP. They include both capital and current
expenditures in the four main sectors: Business enterprise, Government, Higher
education and Private non-profit. R&D covers basic research, applied research,
and experimental development.

Statistical concept and methodology : The gross domestic expenditure on R&D
indicator consists of the total expenditure (current and capital) on R&D by all
resident companies, research institutes, university and government laboratories,
etc. It excludes R&D expenditures financed by domestic firms but performed
abroad. 

The OECD's Frascati Manual defines research and experimental
development as "creative work undertaken on a systemic basis in order to
increase the stock of knowledge, including knowledge of man, culture and
society, and the use of this stock of knowledge 

# Questions <a id='questions'></a>


# Data Wrangling <a id='wrangling'></a>

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

## General Properties

In [7]:
def get_datapoint(path, indicator, dim = 'geo--time'):
  """Reads a datapoint for a given indicator and dimensions.

  Args:
    path : A path to Gapminder World Development Indicator repository.
    indicator : A concept name of an indicator.
    dim : Required dimensions separeted by '--'.

  Returns:
    A pandas.DataFrame indexed by dimensions (or prints non-existent path).
  """

  # Compile the path to CSV file
  path = path + 'datapoints/ddf--datapoints--' + indicator + '--by--' + dim + '.csv'
  index_col = dim.split('--')
  # Either a read csv file or print the incorrect path to check
  try:
    df = pd.read_csv(path, index_col=index_col)
    print(f'Loading {indicator} ...')
    print('Shape: ', df.shape)
    print('===== Head of a DataFrame ====')
    print(df.head(), '\n==============================\n')
    return df
  except:
    print(f'No such file at path\n{path}')
    return None


# Load research and development data
df_rad_exp = get_datapoint(path, 'gb_xpd_rsdv_gd_zs')

# Load GDP growth data
df_gdp_gr = get_datapoint(path, 'ny_gdp_mktp_kd_zg')

# Concatenate R&D data with GDP growth data where both values are available
df = pd.concat([df_gdp_gr, df_rad_exp], axis=1, join='inner').reset_index()

# Shorten names of indicators
df.rename(columns={'ny_gdp_mktp_kd_zg': 'gdp_gr', 'gb_xpd_rsdv_gd_zs': 'rad_exp'}, inplace=True)

# Load country data
countries = pd.read_csv(path + 'ddf--entities--geo--country.csv', index_col='country')

# Add country data into the dataframe
for col in ['name', 'income_groups', 'world_4region']:
  df[col] = df.geo.apply(lambda c: countries.at[c, col])

# Check duplicates and missing values
assert df.duplicated().sum() == 0
print('No duplicated observations.')
assert df.isna().any().any() == False
print('No missing values.')
print('Shape after concatenation and extension :', df.shape)
print('Resulting data set including country data:')
df.head()

Loading gb_xpd_rsdv_gd_zs ...
Shape:  (1527, 1)
===== Head of a DataFrame ====
          gb_xpd_rsdv_gd_zs
geo time                   
alb 2007            0.08737
    2008            0.15412
are 2011            0.48920
    2014            0.70000
arg 1996            0.41749 

Loading ny_gdp_mktp_kd_zg ...
Shape:  (9595, 1)
===== Head of a DataFrame ====
          ny_gdp_mktp_kd_zg
geo time                   
abw 1987           16.07843
    1988           18.64865
    1989           12.12984
    1990            3.96140
    1991            7.96287 

No duplicated observations.
No missing values.
Shape after concatenation and extension : (1524, 7)
Resulting data set including country data:


,geo,time,gdp_gr,rad_exp,name,income_groups,world_4region
0,alb,2007,5.97998,0.08737,Albania,upper_middle_income,europe
1,alb,2008,7.49997,0.15412,Albania,upper_middle_income,europe
2,are,2011,6.93027,0.48920,United Arab Emirates,high_income,asia
3,are,2014,4.28430,0.70000,United Arab Emirates,high_income,asia
4,arg,1996,5.52669,0.41749,Argentina,upper_middle_income,americas


> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

## Data Cleaning (Replace this with more specific notes!)

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


# Exploratory Data Analysis <a id='eda'></a>

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

## Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.

# Generate descriptive statistics
df.describe()

,time,gdp_gr,rad_exp
count,1524.000000,1524.000000,1524.000000
mean,2005.375328,3.858861,0.945200
std,5.190862,4.173827,0.932892
min,1996.000000,-14.838610,0.005440
25%,2001.000000,1.761755,0.245505
50%,2005.000000,3.854205,0.596610
75%,2010.000000,6.154418,1.342307
max,2014.000000,34.466210,4.407440


## Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


# Conclusions <a id='conclusions'></a>

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!

# Limitation <a id='limitation'></a>
